In [5]:
#Imports
from random import shuffle

In [6]:
#Card class
class Card:
    def __init__(self):
        self.suit = None
        self.value = None
    
    def displayValue(self):
        return f"|{self.value} of {self.suit}|"

#Deck class
class Deck:
    def __init__(self):
        self.deck = None
    
    def initializeDeck(self): #Returns list of card objects
        deck = []
        values = ['A','2','3','4','5','6','7','8','9','10','J','K','Q']
        suits = ['Spades','Clubs','Diamonds','Hearts']
        for suit in suits:
            for value in values:
                cardObj = Card()
                cardObj.suit = suit
                cardObj.value = value
                deck.append(cardObj)
        shuffle(deck)
        self.deck = deck
        return None
    
#     def shuffleDeck(self):
#         self.deck = shuffle(self.deck)
#         return None # do not return deck directly

    def isEmpty(self):
        return len(self.deck) == 0
    
    def drawCard(self):
        #drawCard() returns the card and pops the card from the list
        if self.isEmpty():
            print("Deck is empty, no more cards to draw from")
            return  None
        else:
            #Make first item in deck to be the topmost card
            #ie, index 0 in a list [A,B,C] so A is the topmost card\
            return self.deck.pop(0)
    
    def displayDeck(self):
        for card in self.getDeck():
            card.displayValue()
        return None

In [7]:
#Player class
class Player:
    def __init__(self,name="Bob"):
        self.name = name
        self.cards = []
        
    def drawCard(self,deck): #deck is a parameter so that the obj knows where to draw the card from
        self.cards.append(deck.drawCard())
        return None
    
    def displayCards(self):
        if len(self.cards) == 0:
            print(f"{self.name} has no cards")
            return None
        else:
            print(f"{self.name} has cards ",end='')
            for card in self.cards:
                print(card.displayValue(),end=' ')
            print()
            return None
    
    def getValue(self):
        cardMinSet = {'A':1,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'J':10,'Q':10,'K':10}
        cardMaxSet = {'A':11,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'J':10,'Q':10,'K':10}
        minVal,maxVal = 0,0
        for card in self.cards: #Each card in self.cards is a Card() object
            minVal += cardMinSet[card.value]
            maxVal += cardMaxSet[card.value]
        return tuple((minVal,maxVal))
    
    def checkValue(self): #Count the value of the cards held by the player
        cardMinSet = {'A':1,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'J':10,'Q':10,'K':10}
        cardMaxSet = {'A':11,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'J':10,'Q':10,'K':10}
        minVal,maxVal = 0,0
        for card in self.cards: #Each card in self.cards is a Card() object
            minVal += cardMinSet[card.value]
            maxVal += cardMaxSet[card.value]
        if minVal > 21:
            print(f"{self.name} bust! (Value more than {minVal})")
        if maxVal == 21 and len(self.cards) == 2:
            print(f"{self.name} blackjack!")
        elif maxVal == minVal:
            print(f"{self.name}'s value is {maxVal}")
        else:
            print(f"{self.name}'s values are {minVal}(Min) and {maxVal}(Max)")
        return None
    
#Dealer class
class Dealer(Player):
    def __init__(self,name='Dealer'):
        super().__init__(name)
        self.faceUp = None
        #Dealer always has one face up and face down card when 2 cards are drawn per player.
        self.faceDown = None
        
    def drawCard(self,deck):
        if len(self.cards) == 0:
            self.cards.append(deck.drawCard())
            self.faceUp = self.cards[0]
        elif len(self.cards) == 1:
            self.cards.append(deck.drawCard())
            self.faceDown = self.cards[1]
        else:
            self.cards.append(deck.drawCard())
        return None
    
    def displayCards(self):
        if len(self.cards) == 0:
            print(f"{self.name} has no cards")
            return None
        elif len(self.cards) == 2:
            print(f"{self.name} has {self.faceUp.displayValue()}(facedUp), and one unknown card")
        else:
            print(f"{self.name} has cards ",end='')
            for card in self.cards:
                print(card.displayValue(),end=' ')
            print()
            return None
        
    

In [8]:
#Game class
class Game:
    def __init__(self):
        self.playerList = None
        self.dealerWin = 0
        self.playerWin = 0
        self.totalMatch = 0
        self.deck = None
        #Need to calculate total match in because there may be instances of pushes, where player and dealer ends up with the same value
    
    def initializeGame(self):
        #Get number of players, dealer is always single player.
        #noPlayer = int(input("Number of players: ")) #Assume always a valid input
        self.playerList = []
        #Get names of players
        playerNames = ['Bob'] #Currently set to single player
#         playerNames = [name.lstrip().strip() for name in input(("Please name the players, separating with a comma(,) ")).strip().split(',')]
        
        #Dealer is added at the end so that the dealer receives the final card.
        for player in playerNames:
            self.playerList.append(Player(player))
        self.playerList.append(Dealer())
        
        #Create the deck
        self.deck = Deck()
        self.deck.initializeDeck()
        shuffle(self.deck.deck)
        
        return None
        
    
    def displayPlayers(self):
        for player in self.playerList:
            print(player.name)
        return None
    
    def distributeCards(self):
        for _ in range(2): #Draw 2 cards for everyone
            for player in self.playerList:
                player.drawCard(self.deck)
        return None
    
    def displayPlayerCards(self):
        for player in self.playerList:
            player.displayCards()
        return None
    
    def checkWinner(self): #Single player vs Dealer
        #length of self.playerList will be 2, where index 0 is player, and index 1 is the dealer
        dealerVal = self.playerList[1].getValue()
        if max(dealerVal) > 21:
            dealerVal = min(dealerVal)
        else:
            dealerVal = max(dealerVal)
#         print(dealerVal)
        
        playerVal = self.playerList[0].getValue()
        if max(playerVal) > 21:
            playerVal = min(playerVal)
        else:
            playerVal = max(playerVal)
#         print(playerVal)
        
        if dealerVal > playerVal and dealerVal <= 21:
            return 'dealer'
        elif playerVal > dealerVal and playerVal <= 21:
            return 'player'
        elif playerVal > 21:
            return 'dealer'
        elif dealerVal > 21:
            return 'player'
        elif dealerVal > 21 and playerVal > 21:
            return "draw"
        return "draw"
    
    def receiveInput(self): #Manual method of taking in the input from the user
        for player in self.playerList:
            #H-HIT S-STAND 
            #Currently only take in Hit or Stand commands
            #Split/Double/DAS will be implemented after.
            move = input(f"{player.name}'s move: H(hit) S(stand)")
            while move != "S":
                #if not S means H
                player.drawCard(self.deck)
                player.checkValue()
                move = input(f"{player.name}'s move H(hit) S(stand): ")
            if move == "S" and type(player) == Dealer:
                #Check who wins.
                winner = self.checkWinner()
                if winner != "draw":
                    print(f"Winner is {winner}!")
                    if winner == 'dealer':
                        self.dealerWin += 1
                    else:
                        self.playerWin += 1
                else:
                    #Check for bust
                    print("Draw!")
        self.totalMatch += 1
        return None
    
    def getPlayerMove(self): #Auto Input for one player, one dealer.
        #First check if the player has a hard or a soft total
        #self.playerList[0] -> Player
        #self.playerList[1] -> Dealer
        #Since this function is only called after self.distributeCards(), so both
        #Player and Dealer have 2 cards each, where both of player cards and only one of dealer cards is known(faced up)


        #Use a dictionary to map row and card 
        hardTotalMapCol = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, '10': 8, 'A': 9}
        hardTotalMapRow = {'17': 0, '16': 1, '15': 2, '14': 3, '13': 4, '12': 5, '11': 6, '10': 7, '9': 8, '8': 9}
        #hard totals >= 17 always stands.
        #hard totals <= 8 always hits
        hardTotalList = [
            ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
            ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
            ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
            ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
            ['S', 'S', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'S', 'S', 'S', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
        ]

        softTotalMapCol = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, '10': 8, 'A': 9}
        softTotalMapRow = {'A2': 0, 'A3': 1, 'A4': 2, 'A5': 3, 'A6': 4, 'A7': 5, 'A8': 6, 'A9': 7, 'A10': 8}
        softTotalList = [
            ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S'],
            ['S', 'S', 'S', 'S', 'DS', 'S', 'S', 'S', 'S', 'S'],
            [' DS', 'DS', 'DS', 'DS', 'DS', 'S', 'S', 'H', 'H', 'H'],
            ['H', 'D', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'D', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
            ['H', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'H', 'H'],
            ['BJ', 'BJ', 'BJ', 'BJ', 'BJ', 'BJ', 'BJ', 'BJ', 'BJ', 'BJ', 'BJ', ]
        ]

        #Legends
        #S-STAND, H-HIT, DS- DOUBLE if allowed, else stand
        enableDS = False #when DS is disabled, default to stand.

        #Get dealer's faced up card
        dealerUpCard = self.playerList[1].faceUp.value
        if dealerUpCard in 'JQK':
            dealerUpCard = '10'

        #Check player cards
        playerHard = True #playerHard represents if the player card value has a hard or soft total
        for card in self.playerList[0].cards:
            if card.value == 'A':
                playerHard = False #player holds a soft total

        #Get player's total value 
        playerVal = self.playerList[0].getValue() #Tuple in the form of (minVal,maxVal)
        

        #minVal and maxVal is only applicable if the player has an Ace in his hand. Refer to playerHard (bool) to check
        if playerHard: #if player has no ace in his hand, ie having a hard total wehre both vals in tuple are the same
            #Need to first check if the playerValue is > 17 or < 8 because it is not covered by the dict
            playerVal = str(playerVal[0])
            if playerVal <= '17' and playerVal >= '8':
                move = hardTotalList[hardTotalMapRow[playerVal]][hardTotalMapRow[dealerUp]]
            elif playerVal > '17':
                move = 'S'
            else:
                move = 'H'
        else:
            #soft total
            #Need to get the combination of the cards first, ranging from A2-> A10
            #cannot use playerVal because playerVal is a value, and soft totals need you to append the values together.
            playerCards = self.playerList[0].cards
            if playerCards[0].value != 'A':
                playerCards[0],playerCards[1] = playerCards[1],playerCards[0]
            #Swapped the positions to get the correct arrangement
            if playerCards[1].value in 'JQK':
                playerVal = playerCards[0].value + '10'
            else:
                playerVal = playerCards[0].value + playerCards[1].value
            if playerVal == 'AA': #Edge case
                move = 'P'
            else:
                move = softTotalList[softTotalMapRow[playerVal]][softTotalMapCol[dealerUpCard]]
            
            #Determine if the move is a BJ or a DS
            if move == 'DS':
                if enableDS:
                    move = 'H'
                else:
                    move = 'S'
        
        #Move finally obtained through the strat list.

        return move
    
    def getDealerMove(self):
        #First check the dealer's current value
        #Based on blackjack rules, value >= 17, stand else take a card
        #Get dealers value
        dealerVal = self.playerList[1].getValue() #Returns a tuple of two values.
        #If the max value is >= 17, the dealer must stand, or if the ace brings the dealer's value to more than 17
        if dealerVal[1] < 17:
            move = 'H'
        else:
            move = 'S'
        return move
    
    def getAutoInput(self):
        pMove = self.getPlayerMove()
        if pMove == 'H':
            self.playerList[0].drawCard(self.deck)
        if pMove == 'P':
            return None
        dMove = self.getDealerMove()
        if dMove == 'H':
            self.playerList[1].drawCard(self.deck)
        #Make player and dealer move.
        
        return None
        
        

In [9]:
def startRun(count,filename):
    file = open(filename,'w')
    file.write("faceUp,playerVal,winner\n")
    g = Game()
    for _ in range (count):
        g.initializeGame()
        g.distributeCards()
        #Get cards of user, and faced up cards of the dealer.
        dealerFaceUp = g.playerList[1].faceUp.value
        playerValue = g.playerList[0].getValue()
        if playerValue[0] == playerValue[1]:
            #Hard total so both are the same
            playerValue = playerValue[0]
        else:
            #Soft total, need to record in A2-A10 format
            playerCards = g.playerList[0].cards
            if playerCards[0].value != 'A':
                playerCards[0],playerCards[1] = playerCards[1],playerCards[0]
            if playerCards[1].value in 'JQK':
                playerValue = playerCards[0].value + '10'
            else:
                playerValue = playerCards[0].value + playerCards[1].value
            
        g.playerList[0].cards
        g.getAutoInput()
        winner = g.checkWinner()
        file.write(f"{dealerFaceUp},{playerValue},{winner}\n")
        print(f"Face up: {dealerFaceUp}, player value: {playerValue}, winner: {winner}")
    file.close()